In [1]:
from patterns.abstract import *
from patterns.entailment import *
from patterns.contradiction import *
from patterns.neutral import *
from functools import partial
import pandas as pd

In [3]:
se0 = StructuredExplanation('⊆', ('cat', 'animal'))
se1 = StructuredExplanation('∧', (StructuredExplanation('⊆', ('cat', 'animal')), StructuredExplanation('→', ('car', 'vehicle'))))
se2 = StructuredExplanation('∧', (StructuredExplanation('o', ('car', 'vehicle')), StructuredExplanation('⊆', ('cat', 'animal'))))

print(se1 == se2)

False


In [6]:
se2._get_leaves_()

{'animal', 'car', 'cat', 'vehicle'}

In [7]:
eval_data = pd.read_csv("data/cleaned_assigned_samples_training.csv")
gold_label = pd.read_csv("data/assigned_samples_training.csv")

In [77]:
def to_strEx(text: str) -> StructuredExplanation: 
    clauses = text.split('∧')
    strex = []
    for clause in clauses: 
        rel = re.findall("[↔|→|⇒|⊆|]", clause)
        if rel:
            try:
                preds = tuple(s.strip(' .,!?') for s in clause.split(rel[0]))
                strex.append(StructuredExplanation(rel[0], preds))
            except:
                print("error")
    return AbstractPattern.concatenate_explanations(strex)

In [78]:
patterns = [RephrasingPattern(), ImplicationPattern(), EquivalencePattern(), IfThenPattern(), ClassificationPattern()]
get_highlights = lambda n, r: ast.literal_eval(r[f'Sentence1_Highlighted_Ordered_{n}']) + ast.literal_eval(r[f'Sentence2_Highlighted_Ordered_{n}'])
apply_patterns = lambda n, x: [se for se in [pattern(nlp(x[f'Explanation_{n}']), get_highlights(n, x)) for pattern in patterns] if se.relationship != '']
concat = lambda x: AbstractPattern.concatenate_explanations(x)

In [79]:

ent_results = eval_data[eval_data["gold_label"] == "entailment"].copy()

ent_results['Explanation_1_Result'] = ent_results.apply(partial(apply_patterns, 1), axis=1).apply(concat)

ent_output = ent_results[[
    'Explanation_1', 'Explanation_1_Result'
]]

ent_output['gold label'] = gold_label['structured_explanation'].apply(str).apply(str.lower).apply(to_strEx)

ent_output.reset_index(drop=True, inplace=True)

display(ent_output)

/tmp/ipykernel_60514/2619837141.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ent_output['gold label'] = gold_label['structured_explanation'].apply(str).apply(str.lower).apply(to_strEx)


,Explanation_1,Explanation_1_Result,gold label
0,Use an umbrella when it's raining.,,
1,"If the view was from between buses, the woman ...",,
2,You cannot be on a mountain and hiking.,,
3,Going over jump is way to rephrase jumping dir...,,going over jump ↔ jumping dirt ramps
4,The bicyclers are waiting because they are abo...,,
5,Riding a wave means one is outside.,,riding a wave → one is outside
6,Digs into the sand is a way of saying playing ...,,digs into the sand ↔ playing in the sand
7,There is a man sitting in a red chair with ano...,,
8,"Men is plural, which would mean a group. Fris...",frisbee ⊆ game,frisbee ⊆ game
9,A boy playing with a camera means that he has ...,,a boy playing with a camera → he has a camera


In [81]:
not_none = ent_output[ent_output['gold label'].apply(bool)]
not_none['correct'] = not_none['Explanation_1_Result'] == not_none['gold label']
display(not_none)

/tmp/ipykernel_60514/923984470.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_none['correct'] = not_none['Explanation_1_Result'] == not_none['gold label']


,Explanation_1,Explanation_1_Result,gold label,correct
3,Going over jump is way to rephrase jumping dir...,,going over jump ↔ jumping dirt ramps,False
5,Riding a wave means one is outside.,,riding a wave → one is outside,False
6,Digs into the sand is a way of saying playing ...,,digs into the sand ↔ playing in the sand,False
8,"Men is plural, which would mean a group. Fris...",frisbee ⊆ game,frisbee ⊆ game,True
9,A boy playing with a camera means that he has ...,,a boy playing with a camera → he has a camera,False
12,A deck outside is outdoors.,,a deck outside ⊆ outdoors,False
14,If there is a rag tag of the blue bus then the...,rag tag of the blue bus ⇒ there is a blue bus ...,there is a rag tag of the blue bus ⇒ there is ...,False
19,Archery implies shooting arrows.,archery → shooting arrows,archery → shooting arrows,True
20,Jeans are usually made from denim.,,jeans ⊆ denim,False
21,A minivan would usually be found outside.,,minivan → outside,False


In [96]:
accuracy = sum(not_none['correct'])/len(not_none)
print(f"manual accuracy: {accuracy*100:.2f}%")

manual accuracy: 17.65%


In [ ]:
patterns = [NotRephrasingPattern(), NotImplicationPattern(), NotEquivalencePattern(), XORPattern(), IfThenPattern(), NotClassificationPattern()]
get_highlights = lambda n, r: ast.literal_eval(r[f'Sentence1_Highlighted_Ordered_{n}']) + ast.literal_eval(r[f'Sentence2_Highlighted_Ordered_{n}'])
apply_patterns = lambda n, x: [pattern(nlp(x[f'Explanation_{n}']), get_highlights(n, x)) for pattern in patterns]
concat = lambda x: AbstractPattern.concatenate_explanations(x)

In [ ]:

con_results = eval_data[eval_data["gold_label"] == "contradiction"].copy()

con_results['Explanation_1_Result'] = con_results.apply(partial(apply_patterns, 1), axis=1)

con_output = con_results[[
    'Sentence1', 'Sentence2',
    'Sentence1_Highlighted_Ordered_1', 'Sentence2_Highlighted_Ordered_1',
    'Explanation_1', 'Explanation_1_Result',
]]

con_output.reset_index(drop=True, inplace=True)